In [1]:
import torch
from IPython import display
from d2l import torch as d2l
batch_size = 256
train_iter,test_iter = d2l.load_data_fashion_mnist(batch_size)

In [2]:
#数据集有十个类别，所以网络输出维度是10，权重将成一个784*10的矩阵，
num_inputs = 784    #输入是784的向量
num_outputs = 10    
# 使用正态分布初始化权重，偏置初始化为0 
W = torch.normal(0,0.01,size=(num_inputs,num_outputs),requires_grad=True)
b = torch.zeros(num_outputs,requires_grad=True)

In [4]:
X = torch.tensor([[1.0,2.0,3.0],[4.0,5.0,6.0]])
X.sum(0,keepdim=True), X.sum(1,keepdim=True)


(tensor([[5., 7., 9.]]),
 tensor([[ 6.],
         [15.]]))

In [ ]:
def softmax(X):
    X_exp = torch.exp(X)
    partition = X_exp.sum(1,keepdim=True)
    return X_exp / partition

In [ ]:
X = torch.normal(0,1,(2,5))
X_prob = softmax(X)
X_prob, X_prob.sum(1)

(tensor([[0.0825, 0.2439, 0.0878, 0.1640, 0.4218],
         [0.2651, 0.4469, 0.0822, 0.1349, 0.0709]]),
 tensor([1.0000, 1.0000]))

In [ ]:
def net(X):
    return softmax(torch.matmul(X.reshape(-1,W.shape[0]), W) + b)

In [ ]:
y = torch.tensor([0,2])
y_hat = torch.tensor([[0.1,0.3,0.6],[0.3,0.2,0.5]])
y_hat[[0,1],y]
def cross_entropy(y_hat, y ):
    return - torch.log(y_hat[range(len(y_hat)),y])
cross_entropy(y_hat,y)

tensor([2.3026, 0.6931])

In [5]:
def accuracy(y_hat, y):  #@save
    """计算预测正确的数量"""
    # 如果y_hat是一个矩阵(2D数组)，其中每一行代表一组类别分数
    if len(y_hat.shape) > 1 and y_hat.shape[1] > 1:
        #获取每行最大值的索引，将y_hat转换为1D数组，假设y_hat每行对应一个数据点，最大值的索引表示预测类别
        y_hat = y_hat.argmax(axis=1)
    # 逐元素地将得到的1D预测类标签数组与真实标签比较，生成一个布尔数组，预测匹配真实值则为True
    cmp = y_hat.astype(y.dtype) == y
    # 对布尔数组中的True值求和，然后除以总数据量，得到预测正确的比例并转换为浮点数。
    return float(cmp.astype(y.dtype).sum())


In [7]:
class Accumulator:  #@save
    """在n个变量上累加"""
    def __init__(self, n):
        self.data = [0.0] * n

    def add(self, *args):
        self.data = [a + float(b) for a, b in zip(self.data, args)]

    def reset(self):
        self.data = [0.0] * len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]
    
def evaluate_accuracy(net, data_iter):  #@save
    """计算在指定数据集上模型的精度"""
    metric = Accumulator(2)  # 正确预测数、预测总数
    for X, y in data_iter:
        metric.add(accuracy(net(X), y), d2l.size(y))
    return metric[0] / metric[1]